In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
from scipy.spatial.distance import cdist
import time
from tqdm import tqdm
%load_ext autoreload
%autoreload 1
#%aimport __pycache__.lddt.cpython-39
%aimport sautils.encoder_commons
%aimport sautils.common
%aimport torch
import pandas as pd

In [2]:
sid2seq_ss = {}
with open('data/ss.fasta') as file:
    sid = ''
    for line in file:
        line = line.lstrip('>').rstrip('\n')
        if not sid:
            sid = line
        else:
            sid2seq_ss[sid] = line.upper()
            sid = ''
sid2seq_aa = {}
with open('data/aa.fasta') as file:
    sid = ''
    for line in file:
        line = line.lstrip('>').rstrip('\n')
        if not sid:
            sid = line
        else:
            sid2seq_aa[sid] = line.upper()
            sid = ''
mat3di = np.load('data/mat3di.npy') * 2.1
blosum = np.load('data/blosum62.npy') * 1.4
with open('data/mat3di.out') as f:
    lines = f.readlines()
i2a = lines[3].split()
def seq2num(q):
    idx = []
    for a in q:
        idx.append(i2a.index(a))
    return np.array(idx)
    
def calcS_np(ss1,ss2 ,aa1, aa2,mat3di, blosum):
    ss1 = seq2num(ss1)
    ss2 = seq2num(ss2)
    aa1 = seq2num(aa1)
    aa2 = seq2num(aa2)
    comb_array_ss = np.array(np.meshgrid(ss1, ss2)).T.reshape(-1, 2)
    comb_array_aa = np.array(np.meshgrid(aa1, aa2)).T.reshape(-1, 2)
    S = (mat3di[comb_array_ss[:,0],comb_array_ss[:,1]].reshape(len(ss1),-1)) + blosum[comb_array_aa[:,0],comb_array_aa[:,1]].reshape(len(aa1),-1)
    return S
def distances(q_points_aln, t_points_aln):
    d_mat = cdist(q_points_aln, t_points_aln)
    return (d_mat)

In [3]:
coord_dir = 'data/pdb/d*'
if not os.path.exists(coord_dir):
    print('Creating cache first.')
    sautils.encoder_commons.init_coords('data/pdb/d*', coord_dir)
sid2coords = {sid: sautils.encoder_commons.read_coords(coord_dir, sid) for sid in sautils.encoder_commons.list_sids(coord_dir)}

In [5]:
res_0 = pd.read_csv("100vsall/100vsall2/results_0.csv", header = None, delimiter= "\t")
res_1 = pd.read_csv("100vsall/100vsall2/results_15.csv", header = None, delimiter= "\t")
res_2 = pd.read_csv("100vsall/100vsall2/results_30.csv", header = None, delimiter= "\t")
res_3 = pd.read_csv("100vsall/100vsall2/results_45.csv", header = None, delimiter= "\t")
res_4 = pd.read_csv("100vsall/100vsall2/results_60.csv", header = None, delimiter= "\t")
res_5 = pd.read_csv("100vsall/100vsall2/results_75.csv", header = None, delimiter= "\t")
res_6 = pd.read_csv("100vsall/100vsall2/results_90.csv", header = None, delimiter= "\t")
res = [res_0,res_1,res_2,res_3,res_4,res_5,res_6]

In [36]:
len(di_rescale)

1121100

In [34]:
query, target, di_score, rescale_score, di_rescale = [],[],[],[], []
qc, tc = [],[]

In [35]:

for r in res:
    for n,di,rescale in tqdm(zip(r[0].values, r[1].values, r[2].values), total = r[2].values.shape[0]):
        query.append(n[:7])
        target.append(n[8:])
        di_score.append(di)
        rescale_score.append(rescale)
        aa1 = sid2seq_aa[n[:7]]
        aa2 = sid2seq_aa[n[8:]]
        di_rescale.append(di/np.sqrt(len(aa1)*len(aa2)))
        if(type(di/np.sqrt(len(aa1)*len(aa2))) != np.float64):
            print(np.sqrt(len(aa1)*len(aa2)))
        #print(df_result[df_result[0] == n[:7]][1].values)
        #qc.append(df_result[df_result[0] == n[:7]][1].values)
        #tc.append(df_result[df_result[0] == n[8:]][1].values)

100%|███████████████████████████████| 112110/112110 [00:00<00:00, 238978.38it/s]


In [31]:
type(np.sqrt(len(aa1)*len(aa2)))

numpy.float64

In [37]:
df = pd.DataFrame((query,target,di_score,rescale_score, di_rescale)).T

In [38]:
df

,0,1,2,3,4
0,d2jhfa1,d1dlwa_,-26.903744,3.427466,-0.177522
1,d2jhfa1,d2gkma_,-11.406475,2.690378,-0.071931
2,d2jhfa1,d2qrwa_,4.292783,3.44914,0.027178
3,d2jhfa1,d1s69a_,-113.36064,2.746112,-0.726402
4,d2jhfa1,d2bkma_,-119.170235,3.189706,-0.748566
...,...,...,...,...,...
1121095,d1t33a1,d2fk4a1,-39.619259,3.154576,-0.527928
1121096,d1t33a1,d2b9da1,8.330349,2.936015,0.123146
1121097,d1t33a1,d2ewla1,61.953582,3.717077,0.915848
1121098,d1t33a1,d2pkgc1,143.798652,5.079734,1.713833


In [39]:
df.rename(columns={0: 'qname', 1: 'tname'}, inplace=True)

In [40]:
df_result.rename(columns={0: 'tname', 1: 'class'}, inplace=True)

In [41]:
df_m = pd.merge(df, df_result, how = 'inner', on =["tname"])

In [42]:
df_m

,qname,tname,2,3,4,class
0,d2jhfa1,d1dlwa_,-26.903744,3.427466,-0.177522,a.1.1.1
1,d2huga1,d1dlwa_,-8.960694,2.007307,-0.112184,a.1.1.1
2,d1ivha1,d1dlwa_,796.966854,9.542162,6.021751,a.1.1.1
3,d1jyaa_,d1dlwa_,144.797289,4.201377,1.242907,a.1.1.1
4,d1hyua4,d1dlwa_,-64.033086,3.505383,-0.606792,a.1.1.1
...,...,...,...,...,...,...
1121095,d1oota_,d2dkta2,-16.200129,3.949306,-0.284257,g.93.1.1
1121096,d2fmpa2,d2dkta2,-17.683549,2.909457,-0.312995,g.93.1.1
1121097,d1plsa_,d2dkta2,-37.766105,4.052462,-0.474754,g.93.1.1
1121098,d1hc1a3,d2dkta2,30.684665,4.45561,0.262502,g.93.1.1


In [54]:
df = df_m.sort_values(["qname",2], ascending=[False, False])

In [55]:
df.to_csv("./100vsall/100vsall2/results_comb_2.tsv", sep="\t", header=False, index = False)

In [10]:
df_result = pd.read_csv('100vsall/data/scop_lookup.fix.tsv', delimiter='\t', header = None)

In [74]:
df_result

,0,1
0,d1dlwa_,a.1.1.1
1,d2gkma_,a.1.1.1
2,d2qrwa_,a.1.1.1
3,d1s69a_,a.1.1.1
4,d2bkma_,a.1.1.1
...,...,...
11206,d2fk4a1,g.90.1.1
11207,d2b9da1,g.91.1.1
11208,d2ewla1,g.91.1.1
11209,d2pkgc1,g.92.1.1
